In [26]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from pathlib import Path
import xarray as xr
import easygems.healpix as egh
import pandas as pd
import os

In [27]:

# ==== Utility functions ====
def get_nn_lon_lat_point(nside, lon, lat):
    return hp.ang2pix(nside, lon, lat, nest=True, lonlat=True)

def get_pix_indices_for_points(nside, points):
    return [get_nn_lon_lat_point(nside, lon, lat) for lon, lat in points]




In [28]:
# ==== Station info ====
points = [
    (178.453610, -18.147571),  # Suva (Manual)
    (178.453608, -18.147549),  # Suva (AWS)
    (177.438900, -17.618600),  # Lautoka (Manual)
    (177.438730, -17.618897),  # Lautoka (TB3)
    (178.171619, -17.373833),  # Penang (TB3)
    (179.342817, -16.806141),  # Savusavu (Airport)
    (179.339700, -16.468900),  # Labasa (Airport)
]
## name of station 
names = [
    'Suva (Manual)', 'Suva (AWS)', 'Lautoka (Manual)', 'Lautoka (TB3)',
    'Penang (TB3)', 'Savusavu (Airport)', 'Labasa (Airport)'
]


In [29]:
# ==== Parameters ====
zoom_levels = range(3, 11)
temp_res = 1
time_slice = slice('2020-03', '2021-02')
output_dir = "excel_exports"
os.makedirs(output_dir, exist_ok=True)



In [30]:
# ==== Load datasets ====
ds_dict = {}
for zoom in zoom_levels:
    path = f'/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT{temp_res}H.z{zoom}.zarr'
    ds = xr.open_zarr(path)
    ds_dict[zoom] = ds['pr']  # Use the 'pr' variable


In [31]:
# ==== Extract and export ====
for zoom in zoom_levels:
    print(f"Processing zoom level {zoom}...")
    nside = 2 ** zoom
    pix_indices = get_pix_indices_for_points(nside, points)

    pr_um = ds_dict[zoom] * 3600  # Convert to mm/hr
    pr_um_daily = pr_um.sel(time=time_slice)

    for name, pix_index in zip(names, pix_indices):
        pr_point = pr_um_daily.isel(cell=pix_index)
        df = pr_point.to_dataframe(name='value').reset_index()[['time', 'value']]
        
        filename = f"uk_node_{name.replace(' ', '_').replace('(', '').replace(')', '')}_z{zoom}.csv"
        df.to_csv(filename, index=False)

print("✅ All files exported to Excel.")

Processing zoom level 3...
Processing zoom level 4...
Processing zoom level 5...
Processing zoom level 6...
Processing zoom level 7...
Processing zoom level 8...
Processing zoom level 9...
Processing zoom level 10...
✅ All files exported to Excel.
